In [1]:
import os
import pandas as pd
from binance.client import Client
from dotenv import load_dotenv
from datetime import datetime, timezone

# Load API keys
load_dotenv()
API_KEY = os.getenv("BINANCE_API_KEY")
API_SECRET = os.getenv("BINANCE_API_SECRET")

# Initialize Binance Client
client = Client(API_KEY, API_SECRET)
print("Binance API Connected!")

Binance API Connected!


In [7]:
def fetch_historical_data(symbol, interval, start_date, last_date, columns="basic", save_path="data"):
    """Fetch historical OHLCV data from Binance and save to CSV."""
    limit = 1000
    
    print(f"Fetching {symbol} data from {start_date} UTC")
    klines = client.get_historical_klines(symbol, interval, start_date, last_date, limit=limit)
    
    all_columns = [
        'Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume',
        'Close Time', 'Quote Asset Volume', 'Number of Trades',
        'Taker Buy Base Volume', 'Taker Buy Quote Volume', 'Ignore'
    ]
    basic_columns = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
     
    if columns == "basic":
        selected_columns = basic_columns 
    elif columns == "all":
        selected_columns = all_columns

    df = pd.DataFrame(klines, columns=all_columns)
    if columns == "all":
        df['Date'] = pd.to_datetime(df['Timestamp'], unit='ms')
    
    os.makedirs(save_path, exist_ok=True)
    filename = f"{save_path}/{symbol}_{interval}_{start_date}-{last_date}.csv"
    df[selected_columns].to_csv(filename, index=False)
    print(f"Saved data to {filename}")
    
    return df

In [8]:
# Тестируем
df = fetch_historical_data("BTCUSDT", Client.KLINE_INTERVAL_1HOUR, "2023-01-01", "2023-01-02", "all")
df.tail()

Fetching BTCUSDT data from 2023-01-01 UTC
Saved data to data/BTCUSDT_1h_2023-01-01-2023-01-02.csv


,Timestamp,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Volume,Taker Buy Quote Volume,Ignore,Date
20,1672603200000,16603.53000000,16628.00000000,16600.55000000,16604.51000000,4189.05318000,1672606799999,69590383.53779230,147849,2126.61170000,35328801.30161050,0,2023-01-01 20:00:00
21,1672606800000,16604.51000000,16608.83000000,16592.63000000,16606.68000000,3656.77804000,1672610399999,60709003.94278330,136504,1858.75074000,30858901.46496090,0,2023-01-01 21:00:00
22,1672610400000,16607.02000000,16613.85000000,16586.24000000,16602.12000000,3690.19661000,1672613999999,61254180.74634800,126447,1866.90555000,30989586.38986910,0,2023-01-01 22:00:00
23,1672614000000,16602.39000000,16625.00000000,16600.80000000,16616.75000000,3846.61495000,1672617599999,63900690.91129890,128398,1905.95929000,31662349.75208320,0,2023-01-01 23:00:00
24,1672617600000,16617.17000000,16629.05000000,16581.25000000,16588.35000000,4327.06948000,1672621199999,71832138.46919720,149018,2028.47026000,33673522.94941990,0,2023-01-02 00:00:00
